In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Duchenne_Muscular_Dystrophy"
cohort = "GSE109178"

# Input paths
in_trait_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy"
in_cohort_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy/GSE109178"

# Output paths
out_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/GSE109178.csv"
out_gene_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE109178.csv"
out_clinical_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/clinical_data/GSE109178.csv"
json_path = "../../output/preprocess/Duchenne_Muscular_Dystrophy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Asynchronous remodeling is a driver of failed regeneration in Duchenne muscular dystrophy"
!Series_summary	"49 human patient mRNA profiles was generated using HG-U133 Plus 2.0 microarrays. Procesed in Affymetrix Expression console using Plier normalization method and later processed in Partek Genomics Suite. The clustering figure was generated using HCE clustering software."
!Series_summary	"We sought to determine the mechanisms underlying failure of muscle regeneration that is observed in dystrophic muscle through hypothesis generation using muscle profiling data (human dystrophy and murine regeneration). We found that transforming growth factor β-centered networks strongly associated with pathological fibrosis and failed regeneration were also induced during normal regeneration but at distinct time points. We hypothesized that asynchronously regenerating microenvironments are an underlying driver of fibrosis and failed regeneration. We validated

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional, Callable, Dict, Any, List
import json

# 1. Gene Expression Data Availability
# Based on the series title and summary, this is a mRNA profiling study using HG-U133 Plus 2.0 microarrays
# This is likely to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary, we need to identify relevant rows

# 2.1 Trait data - Duchenne Muscular Dystrophy
# From the series overall design, we know there are DMD, BMD, LGMD2I, LGMD2B patients, and normal controls
# Looking at rows 4 and 5, we can see mutation information and pathology notes
# However, we don't have a clear indicator of disease status in a consistent way
# We'll mark trait_row as None since we cannot reliably extract this information
trait_row = None

# 2.2 Age data
# Age data is clearly available in row 0
age_row = 0

# 2.3 Gender data
# Gender data is available in row 3 (labeled as 'Sex')
gender_row = 3

# Data type conversion functions
def convert_trait(value: str) -> Optional[int]:
    # Since we couldn't identify a reliable trait row, this function is defined but won't be used
    if pd.isna(value) or value.lower() == 'na':
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() in ['dmd', 'duchenne muscular dystrophy']:
        return 1
    elif value.lower() in ['control', 'normal', 'healthy']:
        return 0
    return None

def convert_age(value: str) -> Optional[float]:
    if pd.isna(value) or value.lower() == 'na':
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'na':
        return None
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    if pd.isna(value) or value.lower() == 'na':
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    value_lower = value.lower()
    if value_lower in ['m', 'male']:
        return 1
    elif value_lower in ['f', 'female']:
        return 0
    return None

# 3. Save Metadata
# Trait data availability is based on whether trait_row is None
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep according to the instructions


A new JSON file was created at: ../../output/preprocess/Duchenne_Muscular_Dystrophy/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 65
Header line: "ID_REF"	"GSM2934802"	"GSM2934803"	"GSM2934804"	"GSM2934805"	"GSM2934806"	"GSM2934807"	"GSM2934808"	"GSM2934809"	"GSM2934810"	"GSM2934811"	"GSM2934812"	"GSM2934813"	"GSM2934814"	"GSM2934815"	"GSM2934816"	"GSM2934817"	"GSM2934818"	"GSM2934819"	"GSM2934820"	"GSM2934821"	"GSM2934822"	"GSM2934823"	"GSM2934824"	"GSM2934825"	"GSM2934826"	"GSM2934827"	"GSM2934828"	"GSM2934829"	"GSM2934830"	"GSM2934831"	"GSM2934832"	"GSM2934833"	"GSM2934834"	"GSM2934835"	"GSM2934836"	"GSM2934837"	"GSM2934838"	"GSM2934839"	"GSM2934840"	"GSM2934841"	"GSM2934842"	"GSM2934843"	"GSM2934844"	"GSM2934845"	"GSM2934846"	"GSM2934847"	"GSM2934848"	"GSM2934849"	"GSM2934850"
First data line: "1007_s_at"	473.4399	212.6453	279.5325	425.4737	386.6536	337.7515	204.9013	329.2135	464.9574	410.0567	417.8304	466.2427	270.062	305.572	191.3384	325.9095	334.3189	417.6277	447.7753	323.1609	227.914	294.6698	340.4292	305.0576	386.3226	391.1056	506.4699	481.3872	377.7049	264.2037	350.3971	253.458

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the first few gene identifiers:
# - '1007_s_at', '1053_at', '117_at', etc.
# 
# These appear to be Affymetrix probe IDs from microarray platforms, not standard human gene symbols.
# The '_at', '_s_at', etc. suffixes are characteristic of Affymetrix probe identifiers.
# These IDs will need to be mapped to standard human gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation dataframe
# From the gene annotation preview, we can see:
# - 'ID' column contains Affymetrix probe IDs (e.g., '1007_s_at') matching our gene expression data
# - 'Gene Symbol' column contains the human gene symbols we need (e.g., 'DDR1 /// MIR4640')

# 2. Get gene mapping dataframe by extracting the two columns from the gene annotation
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This will handle the many-to-many relationships as specified
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the first few rows of the gene symbol-based expression data
print("Gene symbol-based expression data preview:")
print(gene_data.head(5))

# Show the shape of the resulting gene data
print(f"\nShape of gene expression data: {gene_data.shape}")


Gene symbol-based expression data preview:
           GSM2934802   GSM2934803   GSM2934804   GSM2934805   GSM2934806  \
Gene                                                                        
A1BG        33.077740    39.461580    34.049830    37.717530    26.752070   
A1BG-AS1     4.273378     0.006875     0.463607     7.018493     1.953626   
A1CF        52.088900    48.186680    58.675980    54.752090    61.567710   
A2M       2424.547410  3791.334440  1724.279220  3670.925694  1502.139818   
A2M-AS1    135.953200   125.331400    84.422670    99.955820    95.923650   

           GSM2934807   GSM2934808   GSM2934809   GSM2934810   GSM2934811  \
Gene                                                                        
A1BG        31.816770    28.257260    51.161030    39.648150    42.369040   
A1BG-AS1     6.841659     0.006121     2.195641     2.093436     6.997340   
A1CF        63.544720    59.567480    52.902680    58.037490    47.661880   
A2M       2050.434442  1908.9832

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 49)
First few genes with their expression values after normalization:
           GSM2934802   GSM2934803   GSM2934804   GSM2934805   GSM2934806  \
Gene                                                                        
A1BG        33.077740    39.461580    34.049830    37.717530    26.752070   
A1BG-AS1     4.273378     0.006875     0.463607     7.018493     1.953626   
A1CF        52.088900    48.186680    58.675980    54.752090    61.567710   
A2M       2424.547410  3791.334440  1724.279220  3670.925694  1502.139818   
A2M-AS1    135.953200   125.331400    84.422670    99.955820    95.923650   

           GSM2934807   GSM2934808   GSM2934809   GSM2934810   GSM2934811  \
Gene                                                                        
A1BG        31.816770    28.257260    51.161030    39.648150    42.369040   
A1BG-AS1     6.841659     0.006121     2.195641     2.093436     6.997340   
A1CF        63.544720    59.567480    52.90268

Normalized gene data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE109178.csv
Trait row is None. Cannot extract trait information from clinical data.
Abnormality detected in the cohort: GSE109178. Preprocessing failed.
Data was determined to be unusable due to missing trait indicators and was not saved
